# TESTING NOTEBOOK

## Install packages

In [ ]:
!pip install scikeras
!pip install split-folders # install the package to split the images in TRAIN and TEST folders
!pip install ann_visualizer # visualize NN architectures

## Import libraries

In [ ]:
# IMPORT TENSORFLOW/KERAS
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions

from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions

from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions

from keras.layers import Input
from keras.models import Sequential
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.utils import to_categorical
from keras.layers.core import Activation
from keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import GlobalMaxPooling2D
from keras.layers.convolutional import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

# TRANSFER LEARNING
from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50

# SCIKIT-LEARN/SCIKERAS
from sklearn import preprocessing
import sklearn.metrics as metrics
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

# IMPORT OTHER MODULES
import os
import glob
import datetime

import fnmatch # to count number of image per folder easily
import random # to select random images from a folder
import numpy as np
import pandas as pd
import splitfolders # needed to split the images in TRAIN and TEST folders
from os import mkdir
from pathlib import Path
from shutil import copyfile # to import "copyfile"
import plotly.figure_factory as ff # for printing the heatmap
from collections import defaultdict
from matplotlib import pyplot as plt
import matplotlib.image as img_mat
from sklearn.decomposition import PCA

# CV2
import cv2
from google.colab.patches import cv2_imshow 

In [ ]:
# SET RANDOM SEED TO GET, AS MUCH AS POSSIBLE, REPRODUCIBLE RESULTS
# USING NUMPY
from numpy.random import seed
seed(42)

# USING TENSORFLOW: IN ADDITION, TENSORFLOW HAS ITS OWN RANDOM NUMBER
# GENERATOR THAT MUST ALSO BE SEEDED BY CALLING THE SET_RANDOM_SEED()
# FUNCTION IMMEDIATELY AFTER THE NUMPY RANDOM NUMBER GENERATOR
tf.random.set_seed(42)

In [ ]:
# Link Google Drive account
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# IMPORT UTILS.PY

# 1.Insert the directory
import sys
sys.path.insert(0,"/content/gdrive/MyDrive/Data_Science_2020-2022/Secondo_anno_Secondo_Semestre/FoDL_Project/Project_Example_Food")

# 2.Import your module or file
import utils

In [ ]:
# CONSTANT
# SCALE_PERCENT = 6 # resizing percentage
SEED_VALUE = 42
num_classes = 9 # number of output classes
batch_size = 16
SIZE = 96  #Resize images --> https://www.researchgate.net/post/Which_Image_resolution_should_I_use_for_training_for_deep_neural_network#:~:text=So%20the%20rule%20of%20thumb,for%20something%20smaller%20and%20easier.

# PATH TO THE DATA
ORIGINAL_DATA = "/content/gdrive/MyDrive/Data_Science_2020-2022/Secondo_anno_Secondo_Semestre/FoDL_Project/Project_Example_Food/ExampleFoodImageDataset"
PREPROCESSED_DATA = "/content/gdrive/MyDrive/Data_Science_2020-2022/Secondo_anno_Secondo_Semestre/FoDL_Project/Project_Example_Food/ExampleFoodImageDatasetPreprocessed"
DIR_TRAIN_TEST_DATA = '/content/gdrive/MyDrive/Data_Science_2020-2022/Secondo_anno_Secondo_Semestre/FoDL_Project/Project_Example_Food/train_test' # path contenente i sottoinsiemi di train, test

# LOAD THE FOLDERS WHERE THE TRAINING/TESTING/VALIDATION DATA ARE STORED
base_path_train = '/content/gdrive/MyDrive/Data_Science_2020-2022/Secondo_anno_Secondo_Semestre/FoDL_Project/Project_Example_Food/train_test/train/'
base_path_test = '/content/gdrive/MyDrive/Data_Science_2020-2022/Secondo_anno_Secondo_Semestre/FoDL_Project/Project_Example_Food/train_test/test/'
base_path_val = '/content/gdrive/MyDrive/Data_Science_2020-2022/Secondo_anno_Secondo_Semestre/FoDL_Project/Project_Example_Food/train_test/val/'

# PATH TO THE DATA
DIR_TRAIN_DATA = '/content/gdrive/MyDrive/Data_Science_2020-2022/Secondo_anno_Secondo_Semestre/FoDL_Project/Project_Example_Food/train_test/train/*' # path contenente i sottoinsiemi di train, test
DIR_TEST_DATA = '/content/gdrive/MyDrive/Data_Science_2020-2022/Secondo_anno_Secondo_Semestre/FoDL_Project/Project_Example_Food/train_test/test/*' # path contenente i sottoinsiemi di train, test
DIR_VAL_DATA = '/content/gdrive/MyDrive/Data_Science_2020-2022/Secondo_anno_Secondo_Semestre/FoDL_Project/Project_Example_Food/train_test/val/*' # path contenente i sottoinsiemi di train, test

In [ ]:
# SETTINGS FOR PRETTIER VISUALIZATION IN GOOGLE COLAB
from IPython.display import Javascript
def resize_colab_cell():
  display(Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'))
get_ipython().events.register('pre_run_cell', resize_colab_cell)

# 7.Evaluate and Test on the test data - SPOSTARE QUESTA PARTE NELL'ULTIMO NOTEBOOK --> 2_FoDL_Food_Image_Classification_Model_Testing.ipynb

Ricorda: devi aggiungere anche la classe "No labels" basandoti su un threshold e citando l'articolo che hai trovato a riferimento: se la probabilità più alta è minore del 50 o 60 % per esempio, allora l'immagine appartiene a nessuna classe! Fai anche dei test a proposito!

For testing and validation dataset: https://galaxyinferno.com/what-is-validation-data-used-for-machine-learning-basics/#:~:text=A%20validation%20data%20set%20is,parameters%20within%20the%20model%20class.

## Load the best Model

In [ ]:
# LOAD SAVED MODEL AND GET PATH FOR AN IMAGE
best_model = load_model("mobilenetV2.h5") 

In [ ]:
# PREDICT USING THE BEST MODEL --> USE best_model INSTEAD OF model_mobilenetv2
# Y_pred = model_mobilenetv2.predict(val_generator, val_generator.n // val_generator.batch_size)
Y_pred = best_model.predict(val_generator, val_generator.n // val_generator.batch_size)

val_preds = np.argmax(Y_pred, axis=1)
val_trues = val_generator.classes
print(val_trues)

# PRINT THE CLASSIFICATION REPORT
print(classification_report(val_trues, val_preds))

**Confusion Matrix**

A confusion matrix is a table that is often used to describe the performance of a classification model (or “classifier”) on a set of test data for which the true values are known.

In [ ]:
# Y_pred = model_mobilenetv2.predict(val_generator, val_generator.n // val_generator.batch_size)
Y_pred = best_model.predict(val_generator, val_generator.n // val_generator.batch_size)

val_preds = np.argmax(Y_pred, axis=1)
val_trues = val_generator.classes

# PRINT THE CONFUSION MATRIX
cm = metrics.confusion_matrix(val_trues, val_preds)
cm

#### Create a heatmap as a confusion matrix

In [ ]:
xx = ['caesar_salad', 'caprese_salad', 'french_fries', 'greek_salad', 'hamburger', 'hot_dog', 'pizza', 'sashimi', 'sushi']
yy = ['caesar_salad', 'caprese_salad', 'french_fries', 'greek_salad', 'hamburger', 'hot_dog', 'pizza', 'sashimi', 'sushi']
# yy = ['sushi', 'sashimi', 'pizza', 'hot_dog', 'hamburger', 'greek_salad', 'french_fries', 'caprese_salad', 'caesar_salad']

In [ ]:
# REFERENCES:
# https://stackoverflow.com/questions/60860121/plotly-how-to-make-an-annotated-confusion-matrix-using-a-heatmap

# change each element of z to type string for annotations
z_text = [[str(yy) for y in xx] for x in cm]

# set up figure 
fig = ff.create_annotated_heatmap(cm, x=xx, y=yy, colorscale='Viridis')

# add title
fig.update_layout(title_text='<i><b>Confusion matrix</b></i>',
                  # xaxis = dict(title='x'),
                  # yaxis = dict(title='x')
                 )

# add custom xaxis title
fig.add_annotation(dict(font=dict(color="black",size=14),
                        x=0.5,
                        y=-0.15,
                        showarrow=False,
                        text="Predicted value",
                        xref="paper",
                        yref="paper"))

# add custom yaxis title
fig.add_annotation(dict(font=dict(color="black",size=14),
                        x=-0.15,
                        y=0.5,
                        showarrow=False,
                        text="Real value",
                        textangle=-90,
                        xref="paper",
                        yref="paper"))

# adjust margins to make room for yaxis title
fig.update_layout(margin=dict(t=50, l=200))

# add colorbar
fig['data'][0]['showscale'] = True
fig.show()

In [ ]:
# GET A NEW IMAGE. IMPLEMENT THE SAME THING AS THE BOSCH PROJECT WHERE YOU HAD MULTIPLE RANDOM IMAGES

caprese = "/content/gdrive/MyDrive/Data_Science_2020-2022/Secondo_anno_Secondo_Semestre/FoDL_Project/Project_Example_Food/train_test/val/caprese_salad/smaller_salad (14).jpg_RESIZED.jpg"
fries = "/content/gdrive/MyDrive/Data_Science_2020-2022/Secondo_anno_Secondo_Semestre/FoDL_Project/Project_Example_Food/train_test/val/french_fries/cropfries19.jpg_RESIZED.jpg"
greek_s = "/content/gdrive/MyDrive/Data_Science_2020-2022/Secondo_anno_Secondo_Semestre/FoDL_Project/Project_Example_Food/train_test/val/greek_salad/smaller_salad (29).jpg_RESIZED.jpg"
hamb = "/content/gdrive/MyDrive/Data_Science_2020-2022/Secondo_anno_Secondo_Semestre/FoDL_Project/Project_Example_Food/train_test/val/hamburger/cropburger26.jpg_RESIZED.jpg"
hotD = "/content/gdrive/MyDrive/Data_Science_2020-2022/Secondo_anno_Secondo_Semestre/FoDL_Project/Project_Example_Food/train_test/val/hot_dog/IMG_20160702_192206244.jpg_RESIZED.jpg"
pzz = "/content/gdrive/MyDrive/Data_Science_2020-2022/Secondo_anno_Secondo_Semestre/FoDL_Project/Project_Example_Food/train_test/val/pizza/crop_pizza26.jpg_RESIZED.jpg"
sash = "/content/gdrive/MyDrive/Data_Science_2020-2022/Secondo_anno_Secondo_Semestre/FoDL_Project/Project_Example_Food/train_test/val/sashimi/crop__sashimi9.jpg_RESIZED.jpg"
sush = "/content/gdrive/MyDrive/Data_Science_2020-2022/Secondo_anno_Secondo_Semestre/FoDL_Project/Project_Example_Food/train_test/val/sushi/sushi_121.jpg_RESIZED.jpg"

In [ ]:
# CREATE A LIST OF ALL THE PATH IMAGES
food_path_list = [caprese,
                  fries,
                  greek_s,
                  hamb,
                  hotD,
                  pzz,
                  sash,
                  sush]

Load multiple images and predict

This section has to be implemented inside the Deployment notebook.

In [ ]:
def load_image(img_path, show=True):

    img = image.load_img(img_path, target_size=(224, 224))
    img_tensor = image.img_to_array(img)                    # (height, width, channels)
    img_tensor = np.expand_dims(img_tensor, axis=0)         # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
    img_tensor /= 255.                                      # imshow expects values in the range [0, 1]

    if show:
        plt.imshow(img_tensor[0])                           
        plt.axis('off')
        plt.show()

    return img_tensor
  
img_path = hotD
new_image = load_image(img_path)

pred = model_mobilenetv2.predict(new_image)
# Generate arg maxes for predictions
print(np.argmax(pred, axis = 1))

pred

In [ ]:
print(test_generator.class_indices)

In [ ]:
pred_test = best_model.predict(new_image)
# Generate arg maxes for predictions
print(np.argmax(pred_test, axis = 1))

pred_test

**QUI DOVRESTI AGGIUNGERE CHE, SE IL VALORE MASSIMO è AL DI SOTTO DEL 50% ALLORA LA PREDIZIONE NON è CORRETTA PER NULLA! E ALLORA SI HA LA "NON CLASSE". RIGUARDA PAPER CCHE TI ERI SALVATO ANCHE SU GITHUB E PROVA ANCHE CON ALTRE IMMAGINI CHE NON SONO CIBO!**

In [ ]:
# LOAD MULTIPLE IMAGES
# STILL TO TEST: DOVRESTI ANCHE OTTENERE DIRETTAMENTE L'IMMAGINE CON LA LABEL DI COSA CORRISPONDE!
pred_output_list = [] # --> list()

for images in food_path_list:
  img_path = images
  new_image = load_image(img_path)
  pred = best_model.predict(new_image)
  # Generate arg maxes for predictions
  output_index = np.argmax(pred[[0]], axis = 1)
  print("Output index is:", output_index)

  pred_output_list.append(output_index[0])

print(pred_output_list)

In [ ]:
[1, 2, 8, 4, 5, 6, 4, 8]

In [ ]:
print(test_generator.class_indices)

# USA QUESTO LINK, UTILISSIMO PER LA PARTE DI PREDIZIONE!!!!

https://stackoverflow.com/questions/43469281/how-to-predict-input-image-using-trained-model-in-keras

# 10.Results Analysis

# 11.Deploy in data app --> chek this out: 
- https://dev.to/code_jedi/how-to-turn-your-python-machine-learning-code-into-a-web-app-2hfc

- https://towardsdatascience.com/how-to-deploy-machine-learning-models-601f8c13ff45

- https://www.youtube.com/watch?v=xl0N7tHiwlw

- https://towardsdatascience.com/how-to-build-an-image-classification-app-using-logistic-regression-with-a-neural-network-mindset-1e901c938355

- https://analyticsindiamag.com/deploy-your-deep-learning-based-image-classification-model-with-streamlit/

# 12.Future Work and Improvements

# Comments:


*   Considera, nella parte finale, di aumentare le immagini ad almeno 1000 per classe (se non addirittura a 2000/3000)



# Prova a cross validare con questo modello, vedi se ottieni risultati migliori!

- https://stackoverflow.com/questions/62341053/validation-accuracy-not-improving

# RICORDATI: pip freeze to list the out the packages installed in your environment:

Follow this:
- https://medium.com/@draj0718/image-classification-and-prediction-using-transfer-learning-3cf2c736589d

- https://towardsdatascience.com/how-to-predict-an-image-with-keras-ca97d9cd4817

- https://medium.com/@nutanbhogendrasharma/image-classification-with-resnet50-model-12f4c79c216b#:~:text=keras%2Fmodels%2F.,other%20to%20form%20a%20network.

# UTILIZZA IL REPORT DI BOSCH COME BASE E DIRETTAMENTE L'OVERLEAF CREANDO UN PROGETTO IN LOCALE E CARICANDO SU GITHUB I RISULTATI INTERMEDI E COMPLETI!

# Check this for explanation of dimension and for explanation of each layer CNN:

- https://towardsdatascience.com/image-classification-with-convolutional-neural-networks-12a7b4fb4c91
- https://stackoverflow.com/questions/45561306/understanding-model-summary-keras
- https://iq.opengenus.org/output-size-of-convolution/#:~:text=Machine%20Learning%20(ML)%20cnn&text=In%20short%2C%20the%20answer%20is,%2F%20(stride%20width)%20%2B%201

# Visualization CNN:

- https://netron.app/
- https://towardsdatascience.com/how-to-visualize-neural-network-architectures-in-python-567cd2aa6d62
- http://alexlenail.me/NN-SVG/LeNet.html
- https://datascience.stackexchange.com/questions/12851/how-do-you-visualize-neural-network-architectures

# For the report:

- https://pyimagesearch.com/2019/10/28/3-ways-to-create-a-keras-model-with-tensorflow-2-0-sequential-functional-and-model-subclassing/

Types of creation of a model in keras
- https://pyimagesearch.com/2019/10/28/3-ways-to-create-a-keras-model-with-tensorflow-2-0-sequential-functional-and-model-subclassing/


USE THIS:
CONVERT IMAGEDATAGENERATOR AS NUMPY ARRAY AND FEED THEM INSIDE GRIDSEARCHCV AS X AND Y:
- https://stackoverflow.com/questions/42284873/assign-imagedatagenerator-result-to-numpy-array

FOLLOWING THEN:
- https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/


CHECK:
- https://stackoverflow.com/questions/47279677/how-use-grid-search-with-fit-generator-in-keras
- https://stackoverflow.com/questions/59023969/grid-search-hyperparameters-for-an-image-classification-model
- https://stackoverflow.com/questions/54078455/grid-search-with-gridsearchcv-scikit-learn-hyperparameter-tuning-using-image

CONVERT IMAGES TO ARRAYS
- https://www.quora.com/How-do-I-load-train-and-test-data-from-the-local-drive-for-a-deep-learning-Keras-model

Cerca online anche altre architetture di CNN differenti e aumenta deepness dell'architettura per arrivare almeno a 10 layers!

- https://towardsdatascience.com/image-classification-with-convolutional-neural-networks-12a7b4fb4c91

Altro tipo di architettura:
- https://www.kdnuggets.com/2022/05/image-classification-convolutional-neural-networks-cnns.html

Try:

- https://medium.com/codex/how-to-tune-hyperparameters-for-better-neural-network-performance-b8f542855d2e
- https://towardsdatascience.com/increase-the-accuracy-of-your-cnn-by-following-these-5-tips-i-learned-from-the-kaggle-community-27227ad39554
- https://medium.com/@navmcgill/k-fold-cross-validation-in-keras-convolutional-neural-networks-835bed559d04

Instead of Max pooling, you can also use fractional pooling:

https://stackoverflow.com/questions/44991470/using-tensorflow-layers-in-keras

Check the paper:
Efficient Processing DNN a tutorial and survery, page 10 for reference.